<a href="https://colab.research.google.com/github/tanvir0188/plant-and-crop-pest-detection/blob/main/ml_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
import zipfile



drive.mount('/content/drive')

with zipfile.ZipFile('/content/drive/My Drive/archive.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')



Mounted at /content/drive


In [3]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

dataset_path = 'content/dataset'